In [107]:
import pandas as pd
import numpy as np

In [824]:
"""
Author: Daiwei Lin
Date: 2019/04/01

ECE657A project

"""

import numpy as np
import pandas as pd
import math
import itertools
import copy
from scipy.stats import chi2_contingency

from sklearn.base import BaseEstimator, ClassifierMixin


class DecisionTreeCHAIDNode(BaseEstimator, ClassifierMixin):
    def __init__(self, max_depth, alpha_merge, alpha_stop, num_features=None, k=5):
        # property
        self.max_depth = max_depth

        # constant
        self.num_features = num_features
        self.alpha_merge = alpha_merge
        self.alpha_stop = alpha_stop
        self.k = k

        # variables
        self.children = list()
        self.isleaf = False

        self.split_val = None
        self.split_feature = None
        self.classification = None
        self.is_num_feature = False

        self.depth = 0

    def is_pure(self, y):
        # check whether the branch is pure() having same class )

        # compare all class label with the class of first row.
        # print(y)
        for i in y:
            if i != y[0]:
                return False
        return True

    def find_split_val(self, data, potential_features):
        # Find best split

        b_feature, b_value, b_score, b_groups = None, None, None, None

        for feature in potential_features:
            b_groups_of_cat, adj_p = self.merge(feature, data)
            #             print("feature:{} adj_p:{}, groups:{}".format(feature,adj_p, b_groups_of_cat))

            if b_score is None or adj_p < b_score:
                if b_groups_of_cat is None:
                    groups = None
                else:
                    groups = self.split(feature, b_groups_of_cat, data)

                b_index, b_value, b_score, b_groups = feature, b_groups_of_cat, adj_p, groups

        return {'index': b_index, 'value': b_value, 'groups': b_groups, 'adj_p': b_score}

    def split(self, feature, groups_of_cat, data):
        # split data according to split criteria

        # create one dic pair for each group in split value (groups_of_cat)
        groups = dict()
        groups_len = len(groups_of_cat)
        for i in range(groups_len):
            groups[i] = list()

        selected_feature = data[feature].tolist()

        for f_idx in range(len(selected_feature)):
            for g_idx in range(groups_len):
                if selected_feature[f_idx] in groups_of_cat[g_idx]:
                    groups[g_idx].append(f_idx)
                    break

        for g_idx, f_idx in groups.items():
            groups[g_idx] = data.iloc[f_idx]

        return groups

    def bonf_adjust(self, p, c, r):
        '''

        :param p: p value
        :param c: number of original categories
        :param r: number of merged categories
        :return: adjusted p value
        '''
        B = 0.0
        for i in range(r):
            B += math.pow(-1, i) * math.pow(r - i, c) / (math.factorial(r) * math.factorial(r - i))
        p * B
        #         print("p {}, c {}, r {}, B {}, adj_p {}".format(p,c,r,B, B*p))
        return p * B

    def rm_empty_col(self, c_table):
        # delete columns with all 0 in a contingency table
        # take NUMPY ARRAY as input

        # print(c_table)
        i = 0
        for col in c_table.T:
            if np.sum(col) == 0:
                # print("delete col {}".format(i))
                c_table = np.delete(c_table, i, axis=1)
                i -= 1
            i += 1
        return c_table

    def merge_single_comb(self, c_table, comb):
        # merge contingency table rows according to single combination
        cat_A = comb[0]
        cat_B = comb[1]
        c_table.loc[cat_A,] = c_table.loc[cat_A,] + c_table.loc[cat_B,]
        c_table = c_table.drop(labels=[cat_B], axis=0)
        c_table = c_table.rename(index={cat_A: str(cat_A) + ',' + str(cat_B)})

        return c_table

    def merge(self, feature, data):
        # find best merge of categories in a given feature
        # Return: b_groups: nested list of merged result
        #         adj_p: adjusted p value of merged result

        contingency_table = pd.crosstab(data[feature], data['class'])
        unique_category = contingency_table.index.tolist()
        #         print("contingency table: \n {}".format(contingency_table))
        #         print("feature:{}, unique_category:{}".format(feature,unique_category))
        b_groups, adj_p = None, None
        c = len(unique_category)

        if c == 1:
            adj_p = 1
            return b_groups, adj_p

        elif c == 2:
            sub_table = self.rm_empty_col(contingency_table.values)
            chi2, p, _, _ = chi2_contingency(sub_table)
            b_groups = list()
            for cat in unique_category:
                b_groups.append([cat])
            adj_p = p
            return b_groups, adj_p

        else:

            while True:

                # Get the best combination with
                # highest p value for all pairs of categories
                b_comb, b_pvalue = None, None
                for comb in itertools.combinations(unique_category, 2):
                    sub_table = contingency_table.loc[list(comb)]
                    sub_table = self.rm_empty_col(sub_table.values)

                    chi2, p, _, _ = chi2_contingency(sub_table)
                    if b_pvalue is None or b_pvalue < p:
                        b_comb, b_pvalue = list(comb), p

                if b_pvalue < self.alpha_merge:
                    # stop if best p is smaller than alpha_merge
                    break
                else:
                    # merge selected combination
                    # print("merge {}".format(b_comb))
                    contingency_table = self.merge_single_comb(contingency_table, b_comb)
                    unique_category = contingency_table.index.tolist()
                    # print("unique category: {}".format(unique_category))
                    # stop merging if only 2 categories left
                    if len(unique_category) <= 2:
                        break

            # Get p value for final merged contigency_table
            chi2, p, _, _ = chi2_contingency(contingency_table)
            # Bonferroni adjustment of p value
            r = len(unique_category)
            adj_p = self.bonf_adjust(p, c, r)
            b_groups = list()
            #             print(unique_category)
            for gr in unique_category:
                b_groups.append(gr.split(','))

        return b_groups, adj_p

    def grow(self, data, depth, potential_features):
        #         print("\ndepth={}, p_features={}".format(depth,potential_features))
        y = data['class'].tolist()
        if self.is_pure(y) or self.max_depth <= depth or len(potential_features) == 0:
            #             print("terminate at depth={}".format(depth))
            self.terminate(y, depth)
            return
        else:

            best_split = self.find_split_val(data, potential_features)

            if best_split['adj_p'] <= self.alpha_stop:
                self.split_val = best_split['value']
                self.split_feature = best_split['index']
                self.is_num_feature = self.split_feature in self.num_features
                # print("\nsplit on feature:{} adj_p={}".format(self.split_feature, best_split['adj_p']))

                for idx, gr in best_split['groups'].items():
                    node = DecisionTreeCHAIDNode(self.max_depth, self.alpha_merge, self.alpha_stop, num_features=self.num_features)
                    new_potential_features = copy.deepcopy(potential_features)

                    if len(self.split_val[idx]) == 1:
                        # print(self.split_val[idx])
                        # print(new_potential_features)
                        new_potential_features.remove(self.split_feature)
                    # print("---- new node with features:{}, split_val:{}".format(new_potential_features,self.split_val[idx]))
                    node.grow(gr, depth + 1, new_potential_features)
                    self.children.append(node)
            else:
                # print("adj_p {} larger than alpha stop".format(best_split['adj_p']))
                self.terminate(y, depth)
                # print("{}X{} < {} gini={}".format(self.depth*' ',self.split_feature+1, self.split_val, best_split['gini']))

        self.depth = depth
        return

    def terminate(self, y, depth):
        # define leaf node
        # most frequent class in the data as class label of this node
        # print("terminate at depth {}".format(depth))
        self.classification = max(set(y), key=y.count)
        self.isleaf = True
        self.depth = depth

    def num_to_cat(self, data, num_features, k, is_train, all_bins=None):
        # Convert numerical features into categorical
        data_discretized = copy.copy(data)
        if is_train:
            # Convert training data
            # using binning (quantile cut into k bins)
            labels = [str(x) for x in range(k)]
            numeric_bins = dict()
            print("numerical features:{}".format(num_features))
            for f in num_features:
                # print("f: {}".format(f))
                cat_col, bins = pd.qcut(data_discretized[f], k, retbins=True, duplicates='drop')
                bins = bins.astype(np.float64)
                bins[0] = -np.inf
                bins[-1] = np.inf
                numeric_bins[f] = bins
                cat_col.cat.categories = labels[:len(bins) - 1]
                data_discretized.loc[:, f] = cat_col.astype(str)
        else:
            # Convert testing data
            # match testing data X into numerical bins
            labels = [str(x) for x in range(k)]
            numeric_bins = dict()
            for f in num_features:
                bins = all_bins[f]
                cat_col = pd.cut(data_discretized[f], bins=bins, labels=labels[:len(bins) - 1])
                data_discretized.loc[:, f] = cat_col.astype(str)

        return data_discretized, numeric_bins

    def fit(self, X, y):
        # grow a tree
        # n = len(X.columns)

        if self.num_features is None:
            self.num_features = []

        potential_features = X.columns.tolist()
        X['class'] = y
        data = X
    
        data_discretized, numeric_bins = self.num_to_cat(data, self.num_features, self.k, is_train=True)

        # only define at root node
        self.numeric_bins = numeric_bins  # dictionary type
        self.default_class = y.iloc[0]
        self.grow(data_discretized, 1, potential_features)
        return self

    ##############
    # Print tree #
    ##############
    def print_tree_iterate(self, numeric_bins):
        if not self.isleaf:
            if self.is_num_feature:
                for i in range(len(self.split_val)):
                    print("{}X{} in range {} ".format(self.depth * ' ', self.split_feature, numeric_bins[self.split_feature]))
                    self.children[i].print_tree_iterate(numeric_bins)
            else:
                for i in range(len(self.split_val)):
                    print("{}X{} = {} ".format(self.depth * ' ', self.split_feature, self.split_val[i]))
                    self.children[i].print_tree_iterate(numeric_bins)
        else:
            print("{}[{}]".format(self.depth * ' ', self.classification))

    def print_tree(self):
        # this function is only defined for root node
        self.print_tree_iterate(self.numeric_bins)

    ##############
    # Prediction #
    ##############
    def predict_iterate(self, row, default_class):

        if self.isleaf:
            # is leaf node
            return self.classification
        else:
            # not leaf node
            # predict categorical feature
#             print("NodeFeature:{}, NodeSplitVal={}".format(self.split_feature,self.split_val))
            for i in range(len(self.split_val)):
                
                if row[self.split_feature] in self.split_val[i]:
                    return self.children[i].predict_iterate(row, default_class)
                
            # if it cannot find such combination of feature categories, return a default class.
            return default_class

    def predict(self, X):
        X_discretized, _ = self.num_to_cat(X, self.num_features, self.k, is_train=False, all_bins=self.numeric_bins)
#         print(X_discretized.head(2))
        prediction = []
        for _, row in X_discretized.iterrows():
            prediction.append(self.predict_iterate(row, self.default_class))

        return np.array(prediction)


# Test on Breast Cancer Dataset

In [712]:
import pandas as pd

In [713]:
#Loading dataset
attributes=["Sample code number","Clump Thickness", "Uniformity of Cell Size", "Uniformity of Cell Shape", "Marginal Adhesion","Single Epithelial Cell Size","Bare Nuclei","Bland Chromatin","Normal Nucleoli","Mitoses","class"]
attributes1=["Sample code number","Clump Thickness", "Uniformity of Cell Size", "Uniformity of Cell Shape", "Marginal Adhesion","Single Epithelial Cell Size","Bare Nuclei","Bland Chromatin","Normal Nucleoli","Mitoses"]
df= pd.read_csv("./dataset/breast-cancer-wisconsin.data",header=None,names=attributes, na_values=['?'])
df.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,class
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2


In [714]:
df['class'].replace(to_replace= 2, value = 0, inplace = True)
df['class'].replace(to_replace= 4, value = 1, inplace = True)
df = df.drop(columns=["Sample code number"])
df.head()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,class
0,5,1,1,1,2,1.0,3,1,1,0
1,5,4,4,5,7,10.0,3,2,1,0
2,3,1,1,1,2,2.0,3,1,1,0
3,6,8,8,1,3,4.0,3,7,1,0
4,4,1,1,3,2,1.0,3,1,1,0


In [715]:
df = df.fillna(df.mode().iloc[0])
df.isnull().sum().sum()

0

In [716]:
str_cols = df.drop(columns=['class','Bare Nuclei']).columns.tolist()
df[str_cols] = df[str_cols].astype('str')
df.head()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,class
0,5,1,1,1,2,1.0,3,1,1,0
1,5,4,4,5,7,10.0,3,2,1,0
2,3,1,1,1,2,2.0,3,1,1,0
3,6,8,8,1,3,4.0,3,7,1,0
4,4,1,1,3,2,1.0,3,1,1,0


In [717]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 10 columns):
Clump Thickness                699 non-null object
Uniformity of Cell Size        699 non-null object
Uniformity of Cell Shape       699 non-null object
Marginal Adhesion              699 non-null object
Single Epithelial Cell Size    699 non-null object
Bare Nuclei                    699 non-null float64
Bland Chromatin                699 non-null object
Normal Nucleoli                699 non-null object
Mitoses                        699 non-null object
class                          699 non-null int64
dtypes: float64(1), int64(1), object(8)
memory usage: 54.7+ KB


In [718]:
from sklearn.model_selection import train_test_split
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)

In [719]:
for x in X_train['Clump Thickness']:
    if type(x) != str:
        print(x)

In [720]:
X_train.columns

Index(['Clump Thickness', 'Uniformity of Cell Size',
       'Uniformity of Cell Shape', 'Marginal Adhesion',
       'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
       'Normal Nucleoli', 'Mitoses'],
      dtype='object')

In [721]:
str_cols

['Clump Thickness',
 'Uniformity of Cell Size',
 'Uniformity of Cell Shape',
 'Marginal Adhesion',
 'Single Epithelial Cell Size',
 'Bland Chromatin',
 'Normal Nucleoli',
 'Mitoses']

In [722]:
num_cols = ['Bare Nuclei']

In [786]:
dt = DecisionTreeCHAIDNode(max_depth=10, alpha_merge=0.05, alpha_stop=0.05, num_features=num_cols, k=10)
dt.fit(X_train, y_train)
dt.print_tree()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


numerical features:['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc']
 Xclass = ['ckd'] 
  [ckd]
 Xclass = ['notckd'] 
  [notckd]


In [724]:
np.sum(dt.predict(X_test) == y_test)/len(y_test)

0.948051948051948

# Test on CKD

In [825]:
df = pd.read_csv("./dataset/chronic_kidney_disease_full.csv", na_values=['?','\t?'])
print(df.shape)
df = df.drop(columns = ['id'])
df.head()

(400, 26)


,'age','bp','sg','al','su','rbc','pc','pcc','ba','bgr',...,'pcv','wbcc','rbcc','htn','dm','cad','appet','pe','ane','class'
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd


In [826]:
df = df.fillna(df.mode().iloc[0])
df.isnull().sum().sum()

0

In [827]:
new_col = []
for col in df.columns:
    col = col.replace("'",'')
    new_col.append(col)
df.columns = new_col

In [828]:
str_cols = []
num_cols = []
for col in df.drop(columns = 'class').columns:
    if df[col].dtype != np.int64 and df[col].dtype != np.float64:
        str_cols.append(col)
    else:
        num_cols.append(col)
    
print("categorical columns: {}\n".format(str_cols))
print("numerical columns: {}\n".format(num_cols))

categorical columns: ['rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane']

numerical columns: ['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc']



In [829]:
X = df.drop(columns='class')
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)

In [830]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 268 entries, 80 to 32
Data columns (total 24 columns):
age      268 non-null float64
bp       268 non-null float64
sg       268 non-null float64
al       268 non-null float64
su       268 non-null float64
rbc      268 non-null object
pc       268 non-null object
pcc      268 non-null object
ba       268 non-null object
bgr      268 non-null float64
bu       268 non-null float64
sc       268 non-null float64
sod      268 non-null float64
pot      268 non-null float64
hemo     268 non-null float64
pcv      268 non-null float64
wbcc     268 non-null float64
rbcc     268 non-null float64
htn      268 non-null object
dm       268 non-null object
cad      268 non-null object
appet    268 non-null object
pe       268 non-null object
ane      268 non-null object
dtypes: float64(14), object(10)
memory usage: 52.3+ KB


In [831]:
dt = DecisionTreeCHAIDNode(max_depth=10, alpha_merge=0.05, alpha_stop=0.05, num_features=num_cols, k=20)
dt.fit(X_train, y_train)
dt.print_tree()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


numerical features:['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc']
 Xhemo in range [  -inf  8.1    9.1    9.8   10.1   10.8   11.11  11.6   12.28  13.
 13.6   14.    14.62  15.    15.1   15.695 16.3   17.       inf] 
  Xpcv in range [ -inf 26.   28.   30.05 32.   34.   36.   39.   41.   42.   43.   45.
 46.   48.   51.   52.     inf] 
   [ckd]
  Xpcv in range [ -inf 26.   28.   30.05 32.   34.   36.   39.   41.   42.   43.   45.
 46.   48.   51.   52.     inf] 
   [notckd]
 Xhemo in range [  -inf  8.1    9.1    9.8   10.1   10.8   11.11  11.6   12.28  13.
 13.6   14.    14.62  15.    15.1   15.695 16.3   17.       inf] 
  Xsg in range [ -inf 1.01  1.015 1.02    inf] 
   [ckd]
  Xsg in range [ -inf 1.01  1.015 1.02    inf] 
   Xal in range [-inf 1.   2.   3.   3.65  inf] 
    [notckd]
   Xal in range [-inf 1.   2.   3.   3.65  inf] 
    [ckd]
 Xhemo in range [  -inf  8.1    9.1    9.8   10.1   10.8   11.11  11.6   12.28  13.
 13.6   14.   

In [832]:
np.sum(dt.predict(X_test) == y_test)/len(y_test)

0.946969696969697

In [833]:
pred = dt.predict(X_test)
for i in range(len(pred)):
    if pred[i] is None:
        print(i)
        print(X_test.iloc[i])

In [822]:
df_row = pd.DataFrame(columns = X_test.iloc[18].index.tolist())
df_row.loc[0] = X_test.iloc[18]
df_row

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,hemo,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane
0,60.0,50.0,1.01,0.0,0.0,normal,normal,notpresent,notpresent,261.0,...,15.0,41.0,4200.0,3.4,yes,no,no,good,no,no


In [823]:
dt.predict(df_row)

  age bp sg al su     rbc      pc         pcc          ba bgr  ... hemo pcv  \
0  12  0  0  0  0  normal  normal  notpresent  notpresent  17  ...   13   8   

  wbcc rbcc  htn  dm cad appet  pe ane  
0    0    1  yes  no  no  good  no  no  

[1 rows x 24 columns]
NodeFeature:hemo, NodeSplitVal=[['0', '1', '2', '3', '4', '5', '6', '7', '8'], ['10', '15', '11', '14', '9'], ['12', '16', '17'], ['13']]
NodeFeature:rbcc, NodeSplitVal=[['10', '11', '13', '6'], ['5', '8']]


array([None], dtype=object)

# draft

In [859]:
def get_tree_properties_iterate(tree, max_depth, n_leaf_node, n_internal_node):
    
    mp = 0
    leaf_node = 0
    internal_node = 0
    
    if tree.depth > max_depth:
            mp = tree.depth
    
    if tree.isleaf:
        # is leaf node
        leaf_node = 1   
    else:
        # not leaf node
        internal_node += 1
        for i in range(len(tree.split_val)):
                md, nl, ni = get_tree_properties_iterate(tree.children[i], max_depth, n_leaf_node, n_internal_node)
                max_depth = md
                n_leaf_node, n_internal_node

        # if it cannot find such combination of feature categories, return a default class.
        print("**** called")
        return mp, leaf_node, internal_node
def get_tree_properties(tree):
    max_depth = 0
    n_leaf_node = 0
    n_internal_node = 0
    max_depth, n_leaf_node, n_internal_node = get_tree_properties_iterate(tree, max_depth, n_leaf_node, n_internal_node)
    return {'max_depth':max_depth, 'n_leaf_node':n_leaf_node, 'n_internal_node':n_internal_node}

In [860]:
get_tree_properties(dt)

TypeError: 'NoneType' object is not iterable

In [861]:
dt.print_tree()

 Xhemo in range [  -inf  8.1    9.1    9.8   10.1   10.8   11.11  11.6   12.28  13.
 13.6   14.    14.62  15.    15.1   15.695 16.3   17.       inf] 
  Xpcv in range [ -inf 26.   28.   30.05 32.   34.   36.   39.   41.   42.   43.   45.
 46.   48.   51.   52.     inf] 
   [ckd]
  Xpcv in range [ -inf 26.   28.   30.05 32.   34.   36.   39.   41.   42.   43.   45.
 46.   48.   51.   52.     inf] 
   [notckd]
 Xhemo in range [  -inf  8.1    9.1    9.8   10.1   10.8   11.11  11.6   12.28  13.
 13.6   14.    14.62  15.    15.1   15.695 16.3   17.       inf] 
  Xsg in range [ -inf 1.01  1.015 1.02    inf] 
   [ckd]
  Xsg in range [ -inf 1.01  1.015 1.02    inf] 
   Xal in range [-inf 1.   2.   3.   3.65  inf] 
    [notckd]
   Xal in range [-inf 1.   2.   3.   3.65  inf] 
    [ckd]
 Xhemo in range [  -inf  8.1    9.1    9.8   10.1   10.8   11.11  11.6   12.28  13.
 13.6   14.    14.62  15.    15.1   15.695 16.3   17.       inf] 
  Xrbcc in range [ -inf 3.1   3.57  3.805 4.1   4.5   4.61  4.8